<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

# Customer Support Ticket Dataset

Here we will load a dataset containing customer support tickets. We are using a modified version of the following [dataset](https://www.kaggle.com/datasets/suraj520/customer-support-ticket-dataset). 

We modified the original dataset by removing data that was not needed for this demonstration. Furthermore, we replaced the purchased product placeholders in the ticket description with the actual product name, to show you in later notebooks, how you can extract this information from text with the help of Exasol.


## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

## Create Customer Support Ticket Table

First we have to create the table to import the customer support dataset into.

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(f"""
    CREATE OR REPLACE TABLE {ai_lab_config.db_schema}.CUSTOMER_SUPPORT_TICKETS (
        TICKET_ID DECIMAL(10,0), 
        CUSTOMER_NAME VARCHAR(100), 
        DATE_OF_PURCHASE DATE, 
        TICKET_SUBJECT VARCHAR(255), 
        TICKET_DESCRIPTION VARCHAR(2000), 
        TICKET_STATUS VARCHAR(50), 
        TICKET_CHANNEL VARCHAR(100)
    );
    """)

## Import Data

We will import the data via Exasols CSV Import statement.

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(f"""
    IMPORT INTO {ai_lab_config.db_schema}.CUSTOMER_SUPPORT_TICKETS
    FROM CSV 
    AT 'https://dut5tonqye28.cloudfront.net/ai_lab/text_ai/'
    FILE 'customer_support_tickets.csv'
    SKIP = 1
    """)

## Inspect the Data

Now we can inspect the data.

In [ ]:
from pandas import option_context
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    df = conn.export_to_pandas(f"""
    SELECT * FROM {ai_lab_config.db_schema}.CUSTOMER_SUPPORT_TICKETS
    LIMIT 10
    """)
    with option_context('display.max_rows', 10, 'display.max_colwidth', 1000):
        display(df)